In [1]:
!pip install -qU langchain-google-genai

In [2]:
import os
os.environ["GOOGLE_API_KEY"] = "****"

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [31]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [5]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg.content

"J'adore programmer.\n"

In [6]:
!pip install mysql-connector-python
!pip install pymysql

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [7]:
import mysql.connector  # For mysql-connector-python
import pymysql
# from langchain.utilities import SQLDatabase
from langchain.sql_database import SQLDatabase

db_user = "root"
db_password = "****"
db_host = "localhost:3306"
db_name = "atliq_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [8]:
!pip install -U langchain langchain_experimental

Defaulting to user installation because normal site-packages is not writeable


In [33]:
from langchain_experimental.sql import SQLDatabaseChain
# from langchain.chains.sql_database.base import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose = True)
response = db_chain.run("If I sell all white tshirts, how much I get? remove ``` and sql from SQLQuery and add SQLResult to the result")
response



> Entering new SQLDatabaseChain chain...
If I sell all white tshirts, how much I get? remove ``` and sql from SQLQuery and add SQLResult to the result
SQLQuery:Question: If I sell all white tshirts, how much I get? remove ``` and sql from SQLQuery and add SQLResult to the result
SQLQuery:
```sql
SELECT SUM(`price` * `stock_quantity`)
FROM `t_shirts`
WHERE `color` = 'White';
```

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '```sql\nSELECT SUM(`price` * `stock_quantity`)\nFROM `t_shirts`\nWHERE `color` = 'W' at line 1")
[SQL: ```sql
SELECT SUM(`price` * `stock_quantity`)
FROM `t_shirts`
WHERE `color` = 'White';
```]
(Background on this error at: https://sqlalche.me/e/20/f405)